In [5]:
import geopandas as gpd
from pathlib import Path
import warnings
import hvplot.xarray
import pandas as pd
import xarray as xr
import rioxarray as rxr
import numpy as np

import networkx as nx
import geonetworkx as gnx

Read in the flow direction, reservoir locations, and path to a directory where the outputs will be saved.

In [7]:
# read in the flow direction file
flow_dir_fn = Path("../../data/cumberland/flow-directions.tif")
res_location_fn = Path("../../data/cumberland/cumberland-stations.csv")
save_dir = Path("../../data/cumberland/regulation_data")
network_dir = Path("../../data/cumberland/regulation_data/")

We will be usign the `generate_forcings_from_rat` function to generate the forcings for the reservoirs.

In [8]:
from resorr.data_prep import generate_forcings_from_rat, generate_network
from resorr.network import ReservoirNetwork
from tqdm.notebook import tqdm

help(generate_forcings_from_rat)

Help on function generate_forcings_from_rat in module resorr.data_prep:

generate_forcings_from_rat(reservoir_network, inflow_dir, storage_change_dir, save_dir, aggregate_freq='daily', rat_output_level='final_outputs')
    Args:
        reservoir_network (gnx.GeoDiGraph or str or pathlib.Path): reservoir network as GeoDiGraph or path to directory containing files generated by generate_network
        inflow_dir (str or Path): path to directory containing inflow files
        storage_change_dir (str or Path): path to directory containing storage change files
        save_dir (str or Path): directory to save regulation data to



This function requires the reservoir network as an argument. We can use the network we created in the previous notebook.

In [9]:
edges_fn = network_dir / "rivreg_network.shp"
nodes_fn = network_dir / "rivreg_network_pts.shp"
G = gnx.read_geofiles(nodes_fn, edges_fn, directed=True)
G

In [10]:
forcings = generate_forcings_from_rat(
    G,
    '../../data/cumberland/unregulated-inflow',
    '../../data/cumberland/satellite-dels',
    '../../data/cumberland/regulation_data',
    rat_output_level='rat_outputs',
    aggregate_freq='daily'
)
forcings

<xarray.Dataset>
Dimensions:                     (time: 3189, node: 8)
Coordinates:
  * time                        (time) datetime64[ns] 2013-01-21 ... 2021-10-14
  * node                        (node) int64 0 1 2 3 4 5 6 7
Data variables:
    theoretical_natural_runoff  (time, node) float64 27.2 6.046 ... nan nan
    storage_change              (time, node) float64 nan nan nan ... nan nan nan
    dt                          (time) int64 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1

In [11]:
start_time = pd.to_datetime('2017-01-01')
end_time = pd.to_datetime('2019-01-01')
forcings = forcings.sel(time=slice(start_time, end_time))

reservoir_network = ReservoirNetwork(G, start_time)

for timestep in tqdm(forcings.time.values):
    dt = forcings['dt'].sel(time=timestep).values.item()
    reservoir_network.update(forcings, dt, 'wb')

reservoir_network.data

  0%|          | 0/731 [00:00<?, ?it/s]

<xarray.Dataset>
Dimensions:                     (node: 8, time: 731)
Coordinates:
  * node                        (node) int64 0 1 2 3 4 5 6 7
  * time                        (time) datetime64[ns] 2017-01-01 ... 2019-01-01
Data variables:
    inflow                      (node, time) float64 2.396e+07 ... 1.519e+08
    outflow                     (node, time) float64 0.0 0.0 ... 1.526e+08
    regulated_runoff            (node, time) float64 0.0 0.0 ... 1.193e+08
    natural_runoff              (node, time) float64 2.396e+07 ... 3.268e+07
    theoretical_natural_runoff  (time, node) float64 6.858e+07 ... 1.833e+08
    storage                     (node, time) float64 nan nan nan ... nan nan nan
    storage_change              (time, node) float64 nan nan ... -6.471e+05
    regulation                  (node, time) float64 4.461e+07 ... 3.133e+07
    dt                          (time) int64 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1

In [12]:
reservoir_network.data.to_netcdf('../../data/cumberland/regulation_data/resorr_outputs.nc')

In [22]:
ds = reservoir_network.data

modeled_inflow = ds['inflow'].sel(node=0).hvplot(label='With regulation: Modeled inflow')
tnr = ds['theoretical_natural_runoff'].sel(node=0).hvplot(label='Without regulation: TNR')

modeled_inflow * tnr

:Overlay
   .Curve.With_regulation_colon_Modeled_inflow :Curve   [time]   (inflow)
   .Curve.Without_regulation_colon_TNR         :Curve   [time]   (theoretical_natural_runoff)